In [9]:
import psycopg

connection_params = {
        "host": "localhost",
        "port": 5432,
        "dbname": "mobilitydb",
        "user": "postgres",
        "password": "postgres"
        }

In [10]:
# Define the base function template
base_function_template = """
CREATE OR REPLACE FUNCTION public.psi{index}(
            z integer, x integer, y integer, p_start text, p_end text)
RETURNS bytea
AS $$
    WITH bounds AS (
        SELECT ST_TileEnvelope(z,x,y) as geom
    ),
    trips_ AS (
        SELECT * FROM pymeos_demo LIMIT 100
    )
    ,
    vals AS (
        SELECT mmsi, numInstants(trip) as size, asMVTGeom(transform(attime(trip,span(p_start::timestamptz, p_end::timestamptz, true, true)),3857), transform((bounds.geom)::stbox,3857))
            as geom_times
        FROM (
            SELECT mmsi, trajectory::tgeompoint AS trip
            FROM trips_
        ) as ego, bounds
    ),
    mvtgeom AS (
        SELECT (geom_times).geom, (geom_times).times, size, mmsi
        FROM vals
    )
SELECT ST_AsMVT(mvtgeom) FROM mvtgeom
$$
LANGUAGE 'sql'
STABLE
PARALLEL SAFE;
"""

conn = psycopg.connect(**connection_params)
cursor = conn.cursor()

# Generate 60 copies of the function
for i in range(30):
    # Generate the function with the current index
    function_code = base_function_template.format(index=i*48)
    # Execute the function creation SQL
    cursor.execute(function_code)
    
# Commit the changes
conn.commit()

cursor.close()
conn.close()